## Methane 1


Methane 1のgroファイルを作ってみよう。

In [11]:
import gromacs
import numpy as np


CH = 0.1087  # nm

# 面心立方格子
fcc = (
    np.array(
        [
            [1,1,1],
            [1,0,0],
            [0,1,0],
            [0,0,1]
        ]
    )
    / 2**0.5
)

CC = 0.4
carbons = fcc * CC
cell = np.eye(3) * 2**0.5 * CC
carbons = carbons @ np.linalg.inv(cell)

replicate = np.array([4, 4, 4])
rep_carbons = []
# 座標を3倍に増やす。もとがfractional座標なので、0〜3の範囲になる。
for x in range(replicate[0]):
    for y in range(replicate[0]):
        for z in range(replicate[0]):
            rep_carbons.append(carbons + np.array([x, y, z]))
# 座標を0〜1におさめる
rep_carbons = np.vstack(rep_carbons) / replicate
# セルを大きくする。
# 行ごとに係数をかけたい。
# 多少なりとも読みやすく
rep_cell = np.vstack(
    [cell[0, :] * replicate[0], cell[1, :] * replicate[1], cell[2, :] * replicate[2]]
)
print(rep_cell)

abs_carbons = rep_carbons @ rep_cell

hydrogens = (
    np.array([[-1, -1, -1], [-1, +1, +1], [+1, -1, +1], [+1, +1, -1]]) / 3**0.5 * CH
)


residues = dict()
residues["MET"] = [
    [
        ["C", pos],
        ["H", pos + hydrogens[0]],
        ["H", pos + hydrogens[1]],
        ["H", pos + hydrogens[2]],
        ["H", pos + hydrogens[3]],
    ]
    for pos in abs_carbons
]

frame = gromacs.compose(residues, rep_cell)
with open("Me1.gro", "w") as f:
    gromacs.write_gro(frame, f)

[[2.2627417 0.        0.       ]
 [0.        2.2627417 0.       ]
 [0.        0.        2.2627417]]
